In [1]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    

In [2]:
!pip install ipywidgets==7.7.1
!pip install huggingface_hub
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.1 MB/s eta 0:00:00


In [3]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from transformers import LlamaForCausalLM, LlamaTokenizer
from trl import SFTTrainer
import os

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [5]:
data = load_dataset("code_x_glue_tc_nl_code_search_adv", split="validation")
data_df = data.to_pandas()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9604 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19210 [00:00<?, ? examples/s]

Dataset code_x_glue_tc_nl_code_search_adv downloaded and prepared to /root/.cache/huggingface/datasets/code_x_glue_tc_nl_code_search_adv/default/0.0.0/e55166784c05156ca8363600092a1133f15d75581c07241645d85d5041e1f81a. Subsequent calls will reuse this data.


In [40]:
data_df

,id,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,docstring_summary,parameters,return_statement,argument_list,identifier,nwo,score
0,0,,src/you_get/extractors/dailymotion.py,,,python,"def Func(arg_0, arg_1='.', arg_2=True, arg_3=F...","[def, Func, (, arg_0, ,, arg_1, =, '.', ,, arg...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Downloads Dailymotion videos by URL.,"(url, output_dir='.', merge=True, info_only=Fa...",,,dailymotion_download,soimort/you-get,0.99976
1,1,,src/you_get/extractors/ixigua.py,,,python,"def Func(arg_0):\n """"""Splicing URLs accordi...","[def, Func, (, arg_0, ), :, arg_1, =, [, """", ]...",Splicing URLs according to video ID to get vid...,"[Splicing, URLs, according, to, video, ID, to,...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Splicing URLs according to video ID to get vid...,(video_id),,,get_video_url_from_video_id,soimort/you-get,0.99976
2,2,,src/you_get/common.py,,,python,"def Func(arg_0, arg_1={}, arg_2=True):\n """"...","[def, Func, (, arg_0, ,, arg_1, =, {, }, ,, ar...",Gets the content of a URL via sending a HTTP G...,"[Gets, the, content, of, a, URL, via, sending,...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Gets the content of a URL via sending a HTTP G...,"(url, headers={}, decoded=True)",return data,,get_content,soimort/you-get,0.99976
3,3,,airflow/contrib/hooks/grpc_hook.py,,,python,"def Func(arg_0, arg_1, arg_2=None):\n ""...","[def, Func, (, arg_0, ,, arg_1, ,, arg_2, =, N...","Fetches a field from extras, and returns it. T...","[Fetches, a, field, from, extras, and, returns...",b69c686ad8a0c89b9136bb4b31767257eb7b2597,https://github.com/apache/airflow/blob/b69c686...,"Fetches a field from extras, and returns it. T...","(self, field_name, default=None)",,,GrpcHook._get_field,apache/airflow,0.99944
4,4,,airflow/hooks/postgres_hook.py,,,python,"def Func(arg_0, arg_1, arg_2):\n """"""\n ...","[def, Func, (, arg_0, ,, arg_1, ,, arg_2, ), :...",Dumps a database table into a tab-delimited file,"[Dumps, a, database, table, into, a, tab, -, d...",b69c686ad8a0c89b9136bb4b31767257eb7b2597,https://github.com/apache/airflow/blob/b69c686...,Dumps a database table into a tab-delimited file,"(self, table, tmp_file)",,,PostgresHook.bulk_dump,apache/airflow,0.99944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9599,9599,,environment/lib/python2.7/site-packages/IPytho...,,,python,"def Func(arg_0=None):\n """"""Is the wx event ...","[def, Func, (, arg_0, =, None, ), :, if, arg_0...",Is the wx event loop running.,"[Is, the, wx, event, loop, running, .]",075dc74d1ee62a8c6b7a8bf2b271364f01629d1e,https://github.com/cloud9ers/gurumate/blob/075...,Is the wx event loop running.,(app=None),,,is_event_loop_running_wx,cloud9ers/gurumate,0.00000
9600,9600,,jicbioimage/illustrate/__init__.py,,,python,"def Func(arg_0, arg_1, arg_2, arg_3=(255, 0, 0...","[def, Func, (, arg_0, ,, arg_1, ,, arg_2, ,, a...",Draw a line between pos1 and pos2 on the canva...,"[Draw, a, line, between, pos1, and, pos2, on, ...",d88ddf81ee3eb3949677e2ef746af8169ce88092,https://github.com/JIC-CSB/jicbioimage.illustr...,Draw a line between pos1 and pos2 on the canvas.,"(self, pos1, pos2, color=(255, 0, 0))",,,Canvas.draw_line,JIC-CSB/jicbioimage.illustrate,0.00000
9601,9601,,jicbioimage/illustrate/__init__.py,,,python,"def Func(arg_0, arg_1, arg_2, arg_3=(255, 255,...","[def, Func, (, arg_0, ,, arg_1, ,, arg_2, ,, a...","Write text at x, y top left corner position.\n...","[Write, text, at, x, y, top, left, corner, pos...",d88ddf81ee3eb3949677e2ef746af8169ce88092,https://github.com/JIC-CSB/jicbioimage.illustr...,"Write text at x, y top left corner position.","(self, text, position, color=(255, 255, 255),\...",,,Canvas.text_at,JIC-CSB/jic

In [8]:
data = load_dataset("code_x_glue_tc_nl_code_search_adv", split="test")
test_data = data.to_pandas()

In [39]:
test_data

,id,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,docstring_summary,parameters,return_statement,argument_list,identifier,nwo,score
0,0,,src/you_get/extractors/miomio.py,,,python,"def Func(arg_0):\n """"""str->list\n Conver...","[def, Func, (, arg_0, ), :, arg_1, =, [, ], ar...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,str->list\n Convert XML to URL List.\n F...,(xml_data),return rawurl,,sina_xml_to_url_list,soimort/you-get,0.99976
1,1,,src/you_get/extractors/sina.py,,,python,"def Func(arg_0, arg_1='.', arg_2=True, arg_3=F...","[def, Func, (, arg_0, ,, arg_1, =, '.', ,, arg...",Downloads Sina videos by URL.,"[Downloads, Sina, videos, by, URL, .]",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Downloads Sina videos by URL.,"(url, output_dir='.', merge=True, info_only=Fa...",,,sina_download,soimort/you-get,0.99976
2,2,,src/you_get/util/log.py,,,python,"def Func(arg_0, *arg_1):\n """"""Format text w...","[def, Func, (, arg_0, ,, *, arg_1, ), :, retur...",Format text with color or other effects into A...,"[Format, text, with, color, or, other, effects...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Format text with color or other effects into A...,"(text, *colors)","return ""\33[{}m{content}\33[{}m"".format("";"".jo...",,sprint,soimort/you-get,0.99976
3,3,,src/you_get/util/log.py,,,python,"def Func(arg_0, *arg_1):\n """"""Print a log m...","[def, Func, (, arg_0, ,, *, arg_1, ), :, sys, ...",Print a log message to standard error.,"[Print, a, log, message, to, standard, error, .]",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Print a log message to standard error.,"(text, *colors)",,,print_log,soimort/you-get,0.99976
4,4,,src/you_get/util/log.py,,,python,"dFuncf Func(arg_0, arg_1=NonFunc):\n """"""Pri...","[dFuncf, Func, (, arg_0, ,, arg_1, =, NonFunc,...",Print an error log message.,"[Print, an, error, log, message, .]",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Print an error log message.,"(message, exit_code=None)",,,e,soimort/you-get,0.99976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19205,19205,,jicbioimage/illustrate/__init__.py,,,python,"def Func(arg_0, arg_1):\n """"""Return a b...","[def, Func, (, arg_0, ,, arg_1, ), :, arg_2, =...",Return a blank canvas to annotate.\n\n ...,"[Return, a, blank, canvas, to, annotate, .]",d88ddf81ee3eb3949677e2ef746af8169ce88092,https://github.com/JIC-CSB/jicbioimage.illustr...,Return a blank canvas to annotate.,"(width, height)",return canvas.view(Canvas),,Canvas.blank_canvas,JIC-CSB/jicbioimage.illustrate,0.00000
19206,19206,,jicbioimage/illustrate/__init__.py,,,python,"def Func(arg_0, arg_1, arg_2=(255, 0, 0), arg_...","[def, Func, (, arg_0, ,, arg_1, ,, arg_2, =, (...",Draw a cross on the canvas.\n\n :param ...,"[Draw, a, cross, on, the, canvas, .]",d88ddf81ee3eb3949677e2ef746af8169ce88092,https://github.com/JIC-CSB/jicbioimage.illustr...,Draw a cross on the canvas.,"(self, position, color=(255, 0, 0), radius=4)",,,Canvas.draw_cross,JIC-CSB/jicbioimage.illustrate,0.00000
19207,19207,,jicbioimage/illustrate/__init__.py,,,python,"def Func(arg_0, arg_1=(True, True, True)):\n ...","[def, Func, (, arg_0, ,, arg_1, =, (, True, ,,...",Return a canvas from a grayscale image.\n\n ...,"[Return, a, canvas, from, a, grayscale, image, .]",d88ddf81ee3eb3949677e2ef746af8169ce88092,https://github.com/JIC-CSB/jicbioimage.illustr...,Return a canvas from a grayscale image.,"(im, channels_on=(True, True, True))",return canvas.view(AnnotatedImage),,AnnotatedImage.from_grayscale,JIC-CSB/jicbioimage.illustrate,0.00000
19208,19208,,toolware/utils/generic.py,,,python,"def Func(arg_0=32, arg_1=1):\n """""

In [11]:
!pip install nltk

In [22]:
gound_truth=["def fibonacci_iterative(n): if n <= 0: return -1 fib_series = [0, 1] while len(fib_series) < n: fib_series.append(fib_series[-1] + fib_series[-2]) return fib_series","def calculate_sum(numbers): result = 0 for num in numbers: result += numreturn result"]
generated=["def fibonacci(n): if n == 0: return 0 elif n == 1: return 1 else: return fibonacci(n - 1) + fibonacci(n - 2)","def sum(self, x): return x + 1 def __call__(self, x) return x + 1 def __get__(self, instance, owner):"]

In [23]:
pip install nltk

In [26]:
 import nltk
 nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [37]:
!pip install rouge

In [38]:
def calculate_f1_score(ground_truth, generated):

    total_f1_score = 0

    for reference, candidate in zip(ground_truth, generated):
        f1_score = calculate_f1_score_individual(reference, candidate)
        total_f1_score += f1_score

    num_pairs = len(ground_truth)

    avg_f1_score = total_f1_score / num_pairs

    return avg_f1_score

def calculate_rouge_scores(ground_truth, generated, n=2):

    total_f1_score = 0
    total_rouge_scores = {'ROUGE-1': 0, 'ROUGE-2': 0, 'ROUGE-L': 0}

    for reference, candidate in zip(ground_truth, generated):
        f1_score = calculate_f1_score_individual(reference, candidate)
        total_f1_score += f1_score

        rouge_scores = calculate_ngram_overlap(reference, candidate, n)
        total_rouge_scores['ROUGE-1'] += rouge_scores['ROUGE-1']
        total_rouge_scores['ROUGE-2'] += rouge_scores['ROUGE-2']
        total_rouge_scores['ROUGE-L'] += rouge_scores['ROUGE-L']

    num_pairs = len(ground_truth)

    avg_f1_score = total_f1_score / num_pairs
    avg_rouge_scores = {metric: total_rouge_scores[metric] / num_pairs for metric in total_rouge_scores}

    return avg_f1_score, avg_rouge_scores

# Helper functions for individual scores
def calculate_f1_score_individual(ground_truth, generated):
    reference_tokens = set(ground_truth.split())
    candidate_tokens = set(generated.split())

    common = reference_tokens & candidate_tokens
    precision = len(common) / len(candidate_tokens) if len(candidate_tokens) > 0 else 0
    recall = len(common) / len(reference_tokens) if len(reference_tokens) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1_score

def calculate_ngram_overlap(reference, candidate, n):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    reference_ngrams = [tuple(reference_tokens[i:i + n]) for i in range(len(reference_tokens) - n + 1)]
    candidate_ngrams = [tuple(candidate_tokens[i:i + n]) for i in range(len(candidate_tokens) - n + 1)]

    common_ngrams = set(reference_ngrams) & set(candidate_ngrams)

    precision = len(common_ngrams) / len(candidate_ngrams) if len(candidate_ngrams) > 0 else 0
    recall = len(common_ngrams) / len(reference_ngrams) if len(reference_ngrams) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    rouge_scores = {'ROUGE-1': precision, 'ROUGE-2': precision, 'ROUGE-L': precision}

    return rouge_scores

# Example usage:
ground_truth = [
    "def fibonacci_iterative(n): if n <= 0: return -1 fib_series = [0, 1] while len(fib_series) < n: fib_series.append(fib_series[-1] + fib_series[-2]) return fib_series",
    "def calculate_sum(numbers): result = 0 for num in numbers: result += numreturn result"
]
generated = [
    "def fibonacci(n): if n == 0: return 0 elif n == 1: return 1 else: return fibonacci(n - 1) + fibonacci(n - 2)",
    "def sum(self, x): return x + 1 def __call__(self, x) return x + 1 def __get__(self, instance, owner):"
]

# Calculate average F1 score and average ROUGE scores
avg_f1_score, avg_rouge_scores = calculate_rouge_scores(ground_truth, generated)
print(f"Average F1 Score: {avg_f1_score}")
print(f"Average ROUGE Scores: {avg_rouge_scores}")


Average F1 Score: 0.2101449275362319
Average ROUGE Scores: {'ROUGE-1': 0.045454545454545456, 'ROUGE-2': 0.045454545454545456, 'ROUGE-L': 0.045454545454545456}
